install deps


In [ ]:
%pip install openai
%pip install python-dotenv
%pip install PyPDF2
%pip install transformers
%pip install tiktoken
%pip install beautifulsoup4
%pip install webdriver-manager
%pip install tqdm
%pip install selenium



import libs


In [ ]:
import openai
import json
import os
import dotenv
import PyPDF2
import transformers
import tiktoken
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

dotenv.load_dotenv()

import api key


In [ ]:
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

Test API


In [ ]:
openai.api_key = OPENAI_API_KEY

# list models
models = openai.Model.list()

# print the first model's id
print(models.data[0].id)

# create a chat completion
chat_completion = openai.ChatCompletion.create(
    model="gpt-4", messages=[{"role": "user", "content": "Hello world"}]
)

# print the chat completion
print(chat_completion.choices[0].message.content)

list available models


In [ ]:
# list available models
openai.Engine.list()

pdf loading and parsing


In [ ]:
def extract_pdf_text(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        all_text = ""

        for page_num in range(num_pages):
            page = reader.pages[page_num]
            all_text += page.extract_text()
            all_text += "\n\n"  # Add two newlines between each page

    return all_text


# Replace 'your_file.pdf' with the path to the PDF file you want to read
file_path = "Rowan_Osmon_-_Shell_Hacks.PDF"
extracted_text = extract_pdf_text(file_path)
print(extracted_text)

In [ ]:
extracted_text
print(extracted_text)

check total token count


In [ ]:
from transformers import GPT2Tokenizer


def count_tokens_openai(text):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    encoded_input = tokenizer(text)
    return len(encoded_input["input_ids"])


# Test the function
token_count = count_tokens_openai(extracted_text)
print(f"Total token count: {token_count}")

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained("openai-gpt")

# Tokenize a sentence
tokens = tokenizer(extracted_text)

# Print the tokens
print(tokens)

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


res = num_tokens_from_string(extracted_text, "gpt2")
res

In [ ]:
extracted_text_edited = extracted_text.replace("\n", " ")
message = f"please analyze this resume: {extracted_text_edited}"
message

In [ ]:
chat_completion = openai.ChatCompletion.create(
    model="gpt-4", messages=[{"role": "user", f"content": message}]
)

# print the chat completion
print(chat_completion.choices[0].message.content)

test function calling to extract person details


In [ ]:
messages = [{"role": "user", "content": extracted_text_edited}]
functions = [
    {
        "name": "extractPersonDetails",
        "description": "extract personal details from resume text",
        "parameters": {
            "type": "object",
            "properties": {
                "firstname": {
                    "type": "string",
                    "description": "first name from resume eg john",
                },
                "lastname": {
                    "type": "string",
                    "description": "last name from resume eg doe",
                },
                "email": {
                    "type": "string",
                    "description": "email from resume eg john@gmail.com",
                },
                "phonenumber": {
                    "type": "string",
                    "description": "phone number from resume eg 1234567890",
                },
                "link1": {
                    "type": "string",
                    "description": "link from resume eg linkedin.com/johndoe",
                },
                "link2": {
                    "type": "string",
                    "description": "link from resume eg github.com/johndoe",
                },
                "link3": {
                    "type": "string",
                    "description": "link from resume eg personalwebsite.com/johndoe",
                },
            },
            "required": ["firstname", "lastname", "email", "phonenumber"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]["function_call"]
response_message

In [ ]:
json_text = json.loads(response_message.to_dict()["arguments"])
json_text

AI skill extraction


In [ ]:
messages = [{"role": "user", "content": extracted_text_edited}]
functions = [
    {
        "name": "extractPersonSkills",
        "description": "extract list of skills from resume text",
        "parameters": {
            "type": "object",
            "properties": {
                "skills": {
                    "type": "string",
                    "description": "list of skills from resume eg ['python','java', 'gcp', 'aws']",
                }
            },
            "required": ["skills"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]["function_call"]
response_message

In [ ]:
json_text = json.loads(response_message.to_dict()["arguments"])
json_text = [x.strip() for x in json_text["skills"].split(",")]
json_text

In [ ]:
messages = [{"role": "user", "content": extracted_text_edited}]
functions = [
    {
        "name": "extractPersonEducation",
        "description": "extract list of education from resume text",
        "parameters": {
            "type": "object",
            "properties": {
                "education": {
                    "type": "object",
                    "description": "a json array of education from resume eg [{'name':'FIU', 'degree':'Bachelors of Science', 'major':'Computer Science', 'graduationDate':'2021'}]",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "name of education from resume such as colleges and bootcamps eg FIU, 4geeks",
                        },
                        "degree": {
                            "type": "string",
                            "description": "degree from resume eg Bachelors of Science",
                        },
                        "major": {
                            "type": "string",
                            "description": "major from resume eg Computer Science, business, engineering",
                        },
                        "graduationDate": {
                            "type": "string",
                            "description": "graduation date from resume eg 2021, 2022, 2023",
                        },
                    },
                }
            },
            "required": ["name", "degree", "major", "graduationDate"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]["function_call"]
response_message

In [ ]:
json_text = json.loads(response_message.to_dict()["arguments"])
json_text

In [ ]:
messages = [{"role": "user", "content": extracted_text_edited}]
functions = [
    {
        "name": "extractPersonExperience",
        "description": "extract an array of experiences from resume text",
        "parameters": {
            "type": "object",
            "properties": {
                "experience": {
                    "type": "object",
                    "description": " a json array of experiences from resume eg [{'name':'google', 'title':'software engineer', 'company':'google', 'start':'2021', 'end':'2022', 'description':'built a website using react and python'}], must return the job object in an array",
                    "properties": {
                        "title": {
                            "type": "string",
                            "description": "job title from resume eg software engineer, data scientist",
                        },
                        "company": {
                            "type": "string",
                            "description": "name of company from resume eg google, facebook",
                        },
                        "start": {
                            "type": "string",
                            "description": "start date from resume eg 2021, 2022, 2023 or november 2021, april 2022",
                        },
                        "end": {
                            "type": "string",
                            "description": "end date from resume eg 2021, 2022, 2023 or november 2021, april 2022",
                        },
                        "description": {
                            "type": "string",
                            "description": "job description from resume eg built a website using react and python",
                        },
                    },
                }
            },
            "required": ["title", "company", "start", "end", "description"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]["function_call"]
response_message

In [ ]:
json_text = json.loads(response_message.to_dict()["arguments"])
json_text

In [ ]:
job_url = "https://getwork.com/search/results?keyword=software+engineer&company_ids=&location=&jobId=bc76c8963dd9d02af0335b53567c01bb"

naive, only get initial page load

In [ ]:


def scraper(url):
    try:
        # Fetch the content of the URL using the requests library
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract and return text content (as a demonstration)
        text_content = soup.stripped_strings
        return ' '.join(text_content)
        
    except requests.RequestException as e:
        return f"An error occurred: {e}"

content = scraper(job_url)
print(content)


using a headless web driver

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

def scraper(url):
    try:
        # Initialize headless Chrome
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(options=chrome_options)

        # Navigate to the URL
        driver.get(url)

        # Optional: Wait for JavaScript to load the dynamic content
        # You can set a specific time or wait for a specific element to appear
        # import time
        # time.sleep(5)

        # Get the page source
        page_source = driver.page_source

        # Close the browser
        driver.quit()

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(page_source, 'html.parser')

        # Search for the element by its id
        job_desc_elem = soup.find(id='job-desc')
        # print(job_desc_elem)

        # Extract and return the text content if the element is found
        if job_desc_elem:
            text_content = job_desc_elem.stripped_strings
            return ' '.join(text_content)
        else:
            return "Element with id 'job-desc' not found"

    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
if __name__ == "__main__":
    job_url = job_url
    content = scraper(job_url)
    print(content)


In [ ]:
from datetime import datetime

messages = [{"role": "user", "content": f'currentdate:{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}, job details: {content}'}]
functions = [
    {
        "name": "extractJobDetails",
        "description": "extract job details from job description text, return json object",
        "parameters": {
            "type": "object",
            "properties": {
                "experience": {
                    "type": "object",
                    "description": "",
                    "properties": {
                        "description": {
                            "type": "string",
                            "description": "job description from job description text eg a SaaS company is looking for a software engineer to build a website using react and python",
                        },
                        "responsibilities": {
                            "type": "string",
                            "description": "job responsibilities from job description text eg take requirements from product manager and convert them to business logic",
                        },
                        "requirements": {
                            "type": "string",
                            "description": "job requirements from job description text eg ",
                        },
                        "compensation": {
                            "type": "string",
                            "description": "job compensation from job description text eg $50,000 to $100,000",
                        },
                        "benefits": {
                            "type": "string",
                            "description": "a summary of the job benefits from job description text eg we offer health insurance, dental insurance, and 401k",
                        },
                        "companyvalues": {
                            "type": "string",
                            "description": "a summary of the company values from job description text eg we value diversity, inclusion, and teamwork",
                        },
                        "deadline": {
                            "type": "string",
                            "description": "a deadline for the job application from job description text eg 2021-10-10",
                        },
                        "regularhours": {
                            "type": "string",
                            "description": "Get Job regularhours of job from job description text the type is one of the following: fulltime, parttime, contract, internship, temporary, or volunteer",
                        }
                    },
                }
            },
            "required": ["description", "responsibilities", "requirements", "compensation", "benefits", "companyvalues", "deadline", "type"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]["function_call"]
response_message

In [ ]:
json_text = json.loads(response_message["arguments"])
json_text


In [ ]:
from datetime import datetime

messages = [{"role": "user", "content": f'currentdate:{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}, job details: {content}'}]
functions = [
    {
        "name": "extractJobSkills",
        "description": "extract a list of technical job skills from job description text",
        "parameters": {
            "type": "object",
            "properties": {
                "skills": {
                    "type": "object",
                    "description": "returns a json array of technical job skills from job description text eg [{'name':'python', 'description':'python is a programming language'}]",
                    "properties": {
                        "Skillname": {
                            "type": "string",
                            "description": "technical job skill or requirement from job description text eg python, java, docker, aws, gcp",
                        },
                        "skilldescription": {
                            "type": "string",
                            "description": "a short summarized description of technical job skills from job description and the experience level text eg intermediate python is a programming language",
                        },
                    },
                }
            },
            "required": ["name", "description"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]["function_call"]
response_message

In [ ]:
json_text = json.loads(response_message["arguments"])
json_text

In [ ]:
from datetime import datetime

messages = [{"role": "user", "content": f'currentdate:{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}, job details: {content}'}]
functions = [
    {
        "name": "extractJobLocation",
        "description": "extract job location from job description text",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "object",
                    "description": "returns a details of the jobs location from job description text eg [{'city':'miami', 'state':'florida', 'country':'united states', 'remote':'false'}] ",
                    "properties": {
                        "city": {
                            "type": "string",
                            "description": "extract city from job description text eg miami, new york, san francisco",
                        },
                        "state": {
                            "type": "string",
                            "description": "extract state from job description text eg florida, new york, california",
                        },
                        "country": {
                            "type": "string",
                            "description": "extract country from job description text eg united states, canada, mexico",
                        },
                        "remote": {
                            "type": "string",
                            "description": "extract remote from job description text eg true, false",
                        },
                    },
                }
            },
            "required": ["city", "state", "country", "remote"],
        },
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]["function_call"]
response_message

In [ ]:
json_text = json.loads(response_message["arguments"])
json_text

summary function

In [ ]:
from datetime import datetime

messages = [{"role": "user", "content": content}]
functions = [
    {
        "name": "summary",
        "description": "a long detailed summary of text, include all details",
        "parameters": {
            "type": "object",
            "properties": {
                "summary": {
                    "type": "string",
                    "description": "summary of text",
                    },
                }
            },
            "required": ["city", "state", "country", "remote"],
    }
]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)
response_message = response["choices"][0]["message"]["function_call"]
response_message

In [ ]:
json_text = json.loads(response_message["arguments"])
json_text